# Independent Halting Cascade (IHC) model basic usage
This notebook shows the necessary steps for performing simulations from the IHC model and understanding the model outputs.

As minimal input, the model needs:
- **a network** `edgelist`, represented as a pandas DataFrame with columns 'source' and 'target'
- **recommendation probabilities** `recommendation_probs`: either a number (assigned to all edges) or a vector of size `len(edgelist)`.
- **application probabilities** `application_probs`: either a number (assigned to all nodes) or a vector, one entry per node.
- **hiring probabilities** `hiring_probs`: either a number (assigned to all nodes) or a vector, one entry per node.

To **run the model**, you need to pass a list of `initial spreaders`, and call `model.simulate(initial_spreaders)`.

The output is a *dictionary* with keys:
- `'task_completed'` (Bool): whether the chain was successful (a hire was found) or not
- `'recommenders'` (list of lists): a list with the recommended nodes per step.
- `'applicants'` (list of lists): a list with the applying nodes per step. The first element is always empty because the initial spreader don't apply.
- `'hires'` (list or None): a list with the hired nodes.

In [1]:
import numpy as np
import networkx as nx

# local
import sys
sys.path.append("../src")
from model import IndependentHaltingCascade as IHC
from skills_and_vacancies import sample_application_and_hiring_probs
from network_generators.erdos_renyi import generate_erdos_renyi_edgelist

In [2]:
# network parameters
n = 2000 # nodes
k_avg = 20 # degree
el = generate_erdos_renyi_edgelist(n, avg_degree=k_avg) # erdos-renyi homogeneous network

# IHC probabilities (could be numbers or vectors)
pr = 0.2
pa = 0.5
ph = 0.5

# IHC model
model = IHC(el, recommendation_probs=pr, application_probs=pa, hiring_probs=ph)

# Simulation
initial_spreaders = np.random.choice( model.nodelist.index, 1 ) # choose 1 random node
results = model.simulate(initial_spreaders, return_chain=False)
results

{'task_completed': True,
 'recommenders': [[33], [312]],
 'applicants': [[], [1392, 1673, 1952]],
 'hires': [1392, 1673, 1952]}

In [3]:
# heterogenous case: 
# compute application and hiring probabilities from skill and vacancy parameters
avg_skills = 3
vacancy_reqs = 4
pa_vals, ph_vals = sample_application_and_hiring_probs(n, avg_skills, vacancy_reqs) # pa and ph are vectors (one per node), not numbers

# IHC model
model = IHC(el, recommendation_probs=pr, application_probs=pa_vals, hiring_probs=ph_vals)

# Simulation
initial_spreaders = np.random.choice( model.nodelist.index, 1 ) # choose 1 random node
results = model.simulate(initial_spreaders, return_chain=False)
results

{'task_completed': True,
 'recommenders': [[1553], [294, 98, 707], [1877, 922, 1922, 1502, 1543]],
 'applicants': [[], [], [1268, 1034, 9, 154]],
 'hires': [1034]}